In [1]:
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from libs.code import *
from pytorch_lightning.loggers import TensorBoardLogger
import torch
import pytorch_lightning as pl
import faiss

from libs.Dataset import *
from libs.util import *
from libs.SiameseNetwork import TripletNetworkTask
# non necessari !
# from libs.code import *
# from libs.VAE import *

Imposto i seed e le variabili globali:

In [2]:
random.seed(1996)
np.random.seed(1996)
pl.seed_everything(1996)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Global seed set to 1996


Device: cpu


In [14]:
PATH_DST = 'dataset/all_labels.csv'
PATH_GDRIVE = ''
# TODO: se setto > 0 mi da 
# [W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
# e non mi permette di effettuare il training. tuttavia resta troppo lento. come procedo?
NUM_WORKERS = 0
BATCH_SIZE = 32
NUM_EPOCHS = 10
GPUS = 0
PRETRAINED_MODEL_PATH =  'models/squeezeNet_pretrained.pth'
num_class = 3

# valori pretrained
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225] 

Carico il dataset singolo

In [16]:
transf = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])

dst = TrashbinDataset(csv=PATH_DST, transform=transf)

dst_train, dst_test = split_into_train_and_test(dst)

dst_train_loader = DataLoader(dst_train, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=True)
dst_test_loader = DataLoader(dst_test, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=False)

Estraggo le rappresentazioni rgb dai loader:

In [5]:
dst_train_rep_rgb, dst_train_labels = extract_rgb_representations(loader=dst_train_loader)
dst_test_rep_rgb, dst_test_labels = extract_rgb_representations(loader=dst_test_loader)

100%|██████████| 83/83 [00:43<00:00,  1.90it/s]


Rappresentazioni di training:

In [6]:
dst_train_rep_rgb.shape

(10560, 150528)

Ottengo le predizioni sul test-set usando `predict_nn`:

In [7]:
pred_test_label_rgb = predict_nn(dst_train_rep_rgb, dst_test_rep_rgb, dst_train_labels)
print(f"Sample di label: {pred_test_label_rgb}")

Sample di label: [2 0 0 ... 0 2 2]


Valuto le performance della baseline

In [8]:
classification_error = evaluate_classification(pred_test_label_rgb, dst_test_labels)
print(f"Classification error: {classification_error:0.2f}")

Classification error: 5.29


Importo per effettuare il training della triplenet il miglior modello trovato nella precedente relazione: `SqueezeNet v1.0`. Importo dunque i pesi già trovati dopo il training di 100 epoche .... <b>TODO migliora la descrizione</b> ... importo i pesi.. faccio le opportune modifiche ...

In [9]:
# scarico il modello da pytorch
squeezeNet_1_0 = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)
# applico le opportune modifiche
squeezeNet_1_0.classifier[1] = nn.Conv2d(512, num_class, kernel_size=(1,1), stride=(1,1))
squeezeNet_1_0.num_classes = num_class
# carico i pesi salvati
squeezeNet_1_0.load_state_dict(torch.load(PRETRAINED_MODEL_PATH))

# riduco l'ultimo modello alla funzione attività:
squeezeNet_1_0.classifier = nn.Identity()
squeezeNet_1_0(torch.zeros(1, 3, 224,224)).shape

Using cache found in /Users/danilo/.cache/torch/hub/pytorch_vision_v0.10.0


torch.Size([1, 86528])

In [17]:
# uso il modello, allenato nel precedente progetto, per estrarre le rappresentazioni dal training e dal test set
dst_train_rep, dst_train_labels = extract_rep_squeezeNet(squeezeNet_1_0, dst_train_loader)
dst_test_rep, dst_test_labels = extract_rep_squeezeNet(squeezeNet_1_0, dst_test_loader)

100%|██████████| 83/83 [03:12<00:00,  2.31s/it]


Valuto le performance del sistema:

In [18]:
# valuto le performance del sistema con rappresentazioni non ancora ottimizzate
pred_test_label = predict_nn(dst_train_rep, dst_test_rep, dst_train_labels)
classification_error = evaluate_classification(pred_test_label, dst_test_labels)
print(f"Classification error: {classification_error}")

Classification error: 0.0


Carico il dataset in triplette:

In [19]:
dst_triplet = TripletTrashbin(root=PATH_DST, transform=transf)

dst_train_triplet, dst_test_triplet = split_into_train_and_test(dst_triplet)

triplet_dataset_train_loader = DataLoader(dst_train_triplet, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=True)
triplet_dataset_test_loader = DataLoader(dst_test_triplet, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE)

In [ ]:
# TODO: mostra le immagini delle triplette

Alleno la rete con lr=0.002 che è il migliore trovato per SqueezeNet nel precedente progetto.

In [20]:
triplet_trashbin_task =  TripletNetworkTask(squeezeNet_1_0, lr=0.002)
logger = TensorBoardLogger("metric_logs", name="test_trashbin_v1",)

# TODO: salva ogni ...
# TODO: CALLBACK!!!!!
trainer = pl.Trainer(gpus=GPUS, logger = logger, max_epochs = 10, check_val_every_n_epoch = 5, )
trainer.fit(triplet_trashbin_task, triplet_dataset_train_loader, triplet_dataset_test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name          | Type              | Params
----------------------------------------------------
0 | embedding_net | SqueezeNet        | 735 K 
1 | criterion     | TripletMarginLoss | 0     
----------------------------------------------------
735 K     Trainable params
0         Non-trainable params
735 K     Total params
2.942     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/danilo/GitHub/deep-learning/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:117: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Global seed set to 1996
/Users/danilo/GitHub/deep-learning/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:117: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Epoch 7:  21%|██        | 69/330 [14:03<53:11, 12.23s/it, loss=nan, v_num=0]      

/Users/danilo/GitHub/deep-learning/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:686: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
#TODO: devo estrarre TSNE??
# Come continuo ???